In [6]:
import numpy as np
import matplotlib.pyplot as plt
import csv
import pandas as pd
import sys
import datetime
import time
from pathlib import Path
import json
import psycopg2
import warnings
warnings.filterwarnings('ignore')
import cx_Oracle

## SIP

In [7]:
#Bitte Pfad anpassen für den Ornder bei dir also u-Nummer auswechseln!
path = "C:/users/u216983/SBB/TBET - Documents/06_VU-104_Prozessautomatisierung_J_K/Automatisierung & Predictive Maintenance im Störungsbereich/CSEM_Exports/"
path_to_json = path+"pw_sip.json"

with open(path_to_json, "r") as handler:
    info = json.load(handler)

username = info["username"]
password = info["password"]
service_name = info["service_name"]

#jdbc:oracle:thin:@(DESCRIPTION = (ADDRESS_LIST = (FAILOVER = ON) (LOAD_BALANCE = OFF)(ADDRESS = (PROTOCOL = TCP)(HOST = porasbbmgmscan.sbb.ch)(PORT = 1551)) 
#(ADDRESS = (PROTOCOL = TCP)(HOST = dorasbbmgmscan.sbb.ch oder porasbbmgmscan.sbb.ch)(PORT = 1551))(CONNECT_DATA = (SERVICE_NAME = DWH_PROD)))
con_string = "%s/%s@//SIP2-ODA01-ZK.sbb.ch/%s"%(username, password, service_name)
con = cx_Oracle.connect(con_string)

In [8]:
sip_note = "SELECT * FROM FORM_SM_WF_NOTIZ_1"
df_sip_notiz = pd.read_sql(sip_note, con) # dataframe SIP2.0 Notiz_1

In [10]:
df_sip_notiz.to_csv( 'SIP2_FORM_SM_WF_NOTIZ_1.csv', sep = ',',   encoding='utf-8')# write Datefram to csv file

In [12]:
sip_doc = "SELECT * FROM FORM_SM_DMS_PROCESS_DOC_1"
df_sip_doc = pd.read_sql(sip_doc, con) # dataframe SIP2.0 doc_1

In [17]:
df_sip_doc.to_csv( 'SIP2_FORM_SM_DMS_PROCESS_DOC_1.csv', sep = ',',   encoding='windows-1252')# write Datefram to csv file

In [11]:
sip_worklog = "SELECT * FROM FORM_SM_WF_FS_WORKLOG_1"
df_sip_worklog = pd.read_sql(sip_worklog, con) # dataframe SIP2.0 Notiz_1

In [13]:
df_sip_worklog.to_csv( 'SIP2_FROM FORM_SM_WF_FS_WORKLOG_1.csv', sep = ',',   encoding='utf-8')# write Datefram to csv file

### RAMSI DB

In [8]:
#Verbindung PostgreSql
path = "C:/users/u216983/SBB/TBET - Documents/06_VU-104_Prozessautomatisierung_J_K/Automatisierung & Predictive Maintenance im Störungsbereich/CSEM_Exports/"
path_to_json = path+"pw_postgresql.json"

with open(path_to_json, "r") as handler:
    info = json.load(handler)

    username = info["username"]
    password = info["password"]
    host = info["host"]
    database = info["db"]
    
#establishing the connection

con = psycopg2.connect(host=host, database=database ,user=username, password=password)
#Creating a cursor object using the cursor() method
cur = con.cursor()

#### Table RAMSI_LOADING.FAKT_SIP_TICKET

In [15]:
sip_ticket = cur.execute("SELECT * FROM RAMSI_LOADING.FAKT_SIP_TICKET WHERE C_PROCESSID NOT LIKE 'D%'") # 
data = cur.fetchall()
cols = []
for elt in cur.description:
    cols.append(elt[0])
    
df_sip = pd.DataFrame(data=data, columns=cols) # dataframe SIP Ticket Tabelle

In [16]:
df_sip.to_csv( 'RAMSI_LOADING_FAKT_SIP_TICKETS.csv', sep = ',', encoding='UTF-8')# write Datefram to csv file

#### Table RAMSI_LOADING.FAKT_SIP_WORKLOGS

In [9]:
worklogs = cur.execute("SELECT * FROM RAMSI_LOADING.FAKT_SIP_WORKLOGS")
data = cur.fetchall()
cols = []
for elt in cur.description:
    cols.append(elt[0])
    
df_fs_to_sip =pd.DataFrame(data=data, columns=cols) # dataframe foreign system to SIP

In [11]:
df_fs_to_sip.to_csv( 'RAMSI_LOADING_FAKT_SIP_WORKLOGS.csv', sep = ',', encoding='utf-8')# write Datefram to csv file

#### Table RAMSI_LOADING.FAKT_SIP_EINGRIFFE

In [12]:
eingriffe = cur.execute("SELECT * FROM RAMSI_LOADING.FAKT_SIP_EINGRIFFE")
d = cur.fetchall()
cols = []
for elt in cur.description:
    cols.append(elt[0])
df_sip_eingriffe = pd.DataFrame(d, columns=cols) # dataframe SIP Eingriffe

In [14]:
df_sip_eingriffe.to_csv('RAMSI_LOADING_FAKT_SIP_EINGRIFFE.csv',sep = ',', encoding='utf-8')# write Datefram to csv file

#### Table RAMSI_LOADING.DIM_BETRIEBSPUNKT

In [15]:
operational_location = cur.execute("SELECT * FROM RAMSI_LOADING.DIM_BETRIEBSPUNKT")
data = cur.fetchall()
cols = []
for elt in cur.description:
    cols.append(elt[0])
df_sip_bp = pd.DataFrame(data=data, columns=cols) # dataframe Operational Points Table DIM_BETRIEBSPUNKT (inventory tool)

In [16]:
df_sip_bp.to_csv('RAMSI_LOADING_DIM_BP.csv',sep = ',', encoding='windows-1252')# write Datefram to csv file

#### Table RAMSI_LOADING.DIM_FOREIGN_SYSTEM

In [17]:
foreign_system = cur.execute("SELECT * FROM RAMSI_LOADING.DIM_FOREIGN_SYSTEM")
data = cur.fetchall()
cols = []
for elt in cur.description:
    cols.append(elt[0])
df_dim_fs = pd.DataFrame(data=data, columns=cols) # dataframe Table foreign system from dim (inventory tool)

In [18]:
df_dim_fs.to_csv('RAMSI_LOADING_DIM_FOREIGN_SYSTEM.csv',sep = ',', encoding='windows-1252')# write Datefram to csv file

#### Table RAMSI_LOADING.DIM_SOLUTION

In [19]:
dim_solution = cur.execute("SELECT * FROM RAMSI_LOADING.DIM_SOLUTION")
data = cur.fetchall()
cols = []
for elt in cur.description:
    cols.append(elt[0])
df_dim_solution = pd.DataFrame(data=data, columns=cols) # dataframe Table dim_solution from dim (inventory tool)

In [20]:
df_dim_solution.to_csv('RAMSI_LOADING_DIM_SOLUTION.csv',sep = ',', encoding='windows-1252')# write Datefram to csv file

#### Table RAMSI_LOADING.DIM_DIENSTSTELLE

In [21]:
dim_dienststelle = cur.execute("SELECT * FROM RAMSI_LOADING.DIM_DIENSTSTELLE")
data = cur.fetchall()
cols = []
for elt in cur.description:
    cols.append(elt[0])
df_dim_dienststelle = pd.DataFrame(data=data, columns=cols) # dataframe Table dim_dienststelle from dim (inventory tool)

In [22]:
df_dim_dienststelle.to_csv('RAMSI_LOADING_DIM_DIENSTSTELLE.csv',sep = ',', encoding='windows-1252')# write Datefram to csv file

#### Table RAMSI_LOADING.DIM_MEASURE_1

In [23]:
dim_measure_1 = cur.execute("SELECT * FROM RAMSI_LOADING.DIM_MEASURE_1")
data = cur.fetchall()
cols = []
for elt in cur.description:
    cols.append(elt[0])
df_dim_measure_1 = pd.DataFrame(data=data, columns=cols) # dataframe Table dim_measure_1 from dim (inventory tool)

In [24]:
df_dim_measure_1.to_csv('RAMSI_LOADING_DIM_MEASURE_1.csv',sep = ',', encoding='windows-1252') # write Datefram to csv file

#### Table RAMSI_LOADING.DIM_MEASURE_2

In [25]:
dim_measure_2 = cur.execute("SELECT * FROM RAMSI_LOADING.DIM_MEASURE_2")
data = cur.fetchall()
cols = []
for elt in cur.description:
    cols.append(elt[0])
df_dim_measure_2 = pd.DataFrame(data=data, columns=cols) # dataframe Table dim_measure_2 from dim (inventory tool)

In [26]:
df_dim_measure_2.to_csv('RAMSI_LOADING_DIM_MEASURE_2.csv',sep = ',', encoding='windows-1252') # write Datefram to csv file

#### Table RAMSI_LOADING.DIM_MEASURE_3

In [27]:
dim_measure_3 = cur.execute("SELECT * FROM RAMSI_LOADING.DIM_MEASURE_2")
data = cur.fetchall()
cols = []
for elt in cur.description:
    cols.append(elt[0])
df_dim_measure_3 = pd.DataFrame(data=data, columns=cols) # dataframe Table dim_measure_2 from dim (inventory tool)

In [28]:
df_dim_measure_3.to_csv('RAMSI_LOADING_DIM_MEASURE_3.csv',sep = ',', encoding='windows-1252') # write Datefram to csv file

#### Table RAMSI_LOADING.FAKT_ESQ_EREIGNIS

In [29]:
ereignis = cur.execute("SELECT * FROM RAMSI_LOADING.FAKT_ESQ_EREIGNIS")
data = cur.fetchall()
cols = []
for elt in cur.description:
    cols.append(elt[0])
df_esq_ereignis = pd.DataFrame(data=data, columns=cols) # Datafram Table ESQ Ereignis 

In [30]:
df_esq_ereignis.to_csv('RAMSI_LOADING_FAKT_ESQ_EREIGNIS.csv',sep = ',',encoding='windows-1252') # write Datefram to csv file

#### Table RAMSI_LOADING.DIM_INVENTORYOBJECT

In [31]:
inventory_object = cur.execute("SELECT * FROM RAMSI_LOADING.DIM_INVENTORYOBJECT")
data = cur.fetchall()
cols = []
for elt in cur.description:
    cols.append(elt[0])
df_dim_inventoryobject = pd.DataFrame(data=data, columns=cols) # Datafram Table DIM inventory object 

In [32]:
df_dim_inventoryobject.to_csv('RAMSI_LOADING_DIM_INVENTORYOBJECT.csv',sep = ',', encoding='windows-1252') # write Datefram to csv file

#### Table RAMSI_LOADING.DIM_INV

In [33]:
inv = cur.execute("SELECT * FROM RAMSI_LOADING.DIM_INV")
data = cur.fetchall()
cols = []
for elt in cur.description:
    cols.append(elt[0])
df_dim_inv = pd.DataFrame(data=data, columns=cols) # Datafram Table DIM inventory

In [34]:
df_dim_inv.to_csv('RAMSI_LOADING_DIM_INV.csv',sep = ',',encoding='windows-1252') # write Datefram to csv file

#### Table RAMSI_LOADING.DIM_INV_AMDB_COMPONENT

In [35]:
inv_amdb_component =  cur.execute("SELECT * FROM RAMSI_LOADING.DIM_INV_AMDB_COMPONENT")
data = cur.fetchall()
cols = []
for elt in cur.description:
    cols.append(elt[0])
df_dim_inv_amdb_component = pd.DataFrame(data=data, columns=cols) # Datafram Table DIM component

In [36]:
df_dim_inv_amdb_component.to_csv('RAMSI_LOADING_DIM_INV_AMDB_COMPONENT.csv',sep = ',', encoding='windows-1252') # write Datefram to csv file

#### Table RAMSI_LOADING.DIM_REASON

In [37]:
dim_reason = cur.execute("SELECT * FROM RAMSI_LOADING.DIM_REASON")
data = cur.fetchall()
cols = []
for elt in cur.description:
    cols.append(elt[0])
df_dim_reason = pd.DataFrame(data=data, columns=cols) # Datafram Table DIM inventory object 

In [38]:
df_dim_reason.to_csv('RAMSI_LOADING_DIM_REASON.csv',sep = ',', encoding='windows-1252') # write Datefram to csv file

#### Table RAMSI_LOADING.DIM_TECH_CONSEQ_1

In [39]:
dim_tech_conseq_1 = cur.execute("SELECT * FROM RAMSI_LOADING.DIM_TECH_CONSEQ_1")
data = cur.fetchall()
cols = []
for elt in cur.description:
    cols.append(elt[0])
df_dim_tech_conseq_1 = pd.DataFrame(data=data, columns=cols) # Datafram Table DIM technical consequence

In [40]:
df_dim_tech_conseq_1.to_csv('RAMSI_LOADING_DIM_TECH_CONSEQ_1.csv',sep = ',', encoding='windows-1252') # write Datefram to csv file

#### Table RAMSI_LOADING.DIM_TECH_CONSEQ_2

In [41]:
dim_tech_conseq_2 = cur.execute("SELECT * FROM RAMSI_LOADING.DIM_TECH_CONSEQ_2")
data = cur.fetchall()
cols = []
for elt in cur.description:
    cols.append(elt[0])
df_dim_tech_conseq_2 = pd.DataFrame(data=data, columns=cols) # Datafram Table DIM technical consequence

In [42]:
df_dim_tech_conseq_2.to_csv('RAMSI_LOADING_DIM_TECH_CONSEQ_2.csv',sep = ',', encoding='windows-1252') # write Datefram to csv file

#### Table RAMSI_LOADING.DIM_TECH_CONSEQ_3

In [43]:
dim_tech_conseq_3 = cur.execute("SELECT * FROM RAMSI_LOADING.DIM_TECH_CONSEQ_3")
data = cur.fetchall()
cols = []
for elt in cur.description:
    cols.append(elt[0])
df_dim_tech_conseq_3 = pd.DataFrame(data=data, columns=cols) # Datafram Table DIM technical consequence

In [44]:
df_dim_tech_conseq_3.to_csv('RAMSI_LOADING_DIM_TECH_CONSEQ_3.csv',sep = ',',encoding='windows-1252') # write Datefram to csv file

#### Table RAMSI_LOADING.DIM_STELLWERK

In [45]:
dim_stellwerk = cur.execute("SELECT * FROM RAMSI_LOADING.DIM_STELLWERK")
data = cur.fetchall()
cols = []
for elt in cur.description:
    cols.append(elt[0])
df_dim_stellwerk = pd.DataFrame(data=data, columns=cols) # Datafram Table DIM Stellwerke

In [46]:
df_dim_stellwerk.to_csv('RAMSI_LOADING_DIM_STELLWERK.csv',sep = ',', encoding='windows-1252') # write Datefram to csv file

#### Table RAMSI_LOADING.DIM_OPER_CONSEQ

In [49]:
dim_oper_conseq = cur.execute("SELECT * FROM RAMSI_LOADING.DIM_OPER_CONSEQ")
data = cur.fetchall()
cols = []
for elt in cur.description:
    cols.append(elt[0])
df_dim_oper_conseq = pd.DataFrame(data=data, columns=cols) # Datafram Table operational consequences

In [50]:
df_dim_oper_conseq.to_csv('RAMSI_LOADING_DIM_OPER_CONSEQ.csv',sep = ',', encoding='windows-1252') # write Datefram to csv file

#### Table RAMSI_LOADING.DIM_MILESTONE_PRIORITY

In [51]:
dim_milestone_priority = cur.execute("SELECT * FROM RAMSI_LOADING.DIM_MILESTONE_PRIORITY")
data = cur.fetchall()
cols = []
for elt in cur.description:
    cols.append(elt[0])
df_dim_milestone_priority = pd.DataFrame(data=data, columns=cols) # Datafram Table DIM milstone priority

In [52]:
df_dim_milestone_priority.to_csv('RAMSI_LOADING_DIM_MILESTONE_PRIORITY.csv',sep = ',', encoding='windows-1252') # write Datefram to csv file

#### Table RAMSI_LOADING.DIM_INSIDENT_TYPE

In [53]:
dim_incident_type = cur.execute("SELECT * FROM RAMSI_LOADING.DIM_INCIDENT_TYPE")
data = cur.fetchall()
cols = []
for elt in cur.description:
    cols.append(elt[0])
df_dim_incident_type = pd.DataFrame(data=data, columns=cols) # Datafram Table DIM incident type

In [54]:
df_dim_incident_type.to_csv('RAMSI_LOADING_DIM_INCIDENT_TYPE.csv',sep = ',') # write Datefram to csv file

#### Table RAMSI_LOADING.DIM_LAST_ASSIGNING_OU

In [55]:
dim_assigning_ou = cur.execute("SELECT * FROM RAMSI_LOADING.DIM_LAST_ASSIGNING_OU")
data = cur.fetchall()
cols = []
for elt in cur.description:
    cols.append(elt[0])
df_dim_assigning_ou = pd.DataFrame(data=data, columns=cols) # Datafram Table DIM ASSIGNING

In [56]:
df_dim_assigning_ou.to_csv('RAMSI_LOADING_DIM_LAST_ASSIGNING_OU.csv',sep = ',', encoding='windows-1252') # write Datefram to csv file

#### Table RAMSI_LOADING.DIM_ESI_CAUSE

In [57]:
dim_esi_cause = cur.execute("SELECT * FROM RAMSI_LOADING.DIM_ESI_CAUSE")
data = cur.fetchall()
cols = []
for elt in cur.description:
    cols.append(elt[0])
df_dim_esi_cause = pd.DataFrame(data=data, columns=cols) # Datafram Table DIM ESI CAUSE

In [58]:
df_dim_esi_cause.to_csv('RAMSI_LOADING_DIM_ESI_CAUSE.csv',sep = ',',encoding='windows-1252') # write Datefram to csv file